In [1]:
# load the dataset
from datasets import load_dataset

kr3 = load_dataset("Wittgensteinian/KR3", name='kr3', split='train')
kr3 = kr3.remove_columns(['__index_level_0__'])

Using custom data configuration Wittgensteinian--KR3-9f7e41b8ab859e44
Reusing dataset parquet (/home/gsdsaml/.cache/huggingface/datasets/parquet/Wittgensteinian--KR3-9f7e41b8ab859e44/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901)


In [5]:
kr3_binary = kr3.filter(lambda example: example['Rating'] != 2)

Loading cached processed dataset at /home/gsdsaml/.cache/huggingface/datasets/parquet/Wittgensteinian--KR3-9f7e41b8ab859e44/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901/cache-964db1339d22becb.arrow


In [6]:
kr3_binary[0]

{'Rating': 1,
 'Review': '숙성 돼지고기 전문점입니다. 건물 모양 때문에 매장 모양도 좀 특이하지만 쾌적한 편이고 살짝 레트로 감성으로 분위기 잡아놨습니다. 모든 직원분들께서 전부 가능하다고 멘트 쳐주시며, 고기는 초반 커팅까지는 구워주십니다. 가격 저렴한 편 아니지만 맛은 준수합니다. 등심덧살이 인상 깊었는데 구이로 별로일 줄 알았는데 육향 짙고 얇게 저며 뻑뻑하지 않았습니다. 하이라이트는 된장찌개. 진짜 굿입니다. 버터 간장밥, 골뱅이 국수 등 나중에 더 맛봐야 할 것들은 남겨뒀습니다.'}

In [149]:
# load the attn and logits
import torch
i = 0
attns = torch.load(f'outputs/FT3/attentions/batch_{i}.pt')
logits = torch.load(f'outputs/FT3/logits/batch_{i}.pt')

In [150]:
attns.size(), logits.size()

(torch.Size([32, 12, 184]), torch.Size([32, 2]))

In [151]:
attn_per_token = attns[0][-1]
logit = logits[0]

In [152]:
attn_per_token

tensor([0.0062, 0.0058, 0.0061, 0.0057, 0.0060, 0.0063, 0.0067, 0.0068, 0.0069,
        0.0063, 0.0064, 0.0061, 0.0032, 0.0040, 0.0030, 0.0037, 0.0040, 0.0035,
        0.0037, 0.0026, 0.0033, 0.0032, 0.0026, 0.0035, 0.0052, 0.0026, 0.0056,
        0.0067, 0.0072, 0.0069, 0.0070, 0.0064, 0.0067, 0.0066, 0.0079, 0.0073,
        0.0068, 0.0069, 0.0071, 0.0061, 0.0069, 0.0069, 0.0070, 0.0069, 0.0064,
        0.0063, 0.0039, 0.0046, 0.0037, 0.0074, 0.0063, 0.0060, 0.0060, 0.0067,
        0.0058, 0.0055, 0.0058, 0.0073, 0.0052, 0.0058, 0.0061, 0.0060, 0.0061,
        0.0059, 0.0058, 0.0058, 0.0048, 0.0060, 0.0064, 0.0043, 0.0041, 0.0062,
        0.0067, 0.0070, 0.0060, 0.0065, 0.0057, 0.0035, 0.0027, 0.0056, 0.0049,
        0.0078, 0.0075, 0.0031, 0.0029, 0.0030, 0.0080, 0.0078, 0.0074, 0.0077,
        0.0073, 0.0071, 0.0064, 0.0048, 0.0055, 0.0061, 0.0068, 0.0045, 0.0079,
        0.0086, 0.0084, 0.0058, 0.0029, 0.0030, 0.0027, 0.0026, 0.0026, 0.0026,
        0.0025, 0.0024, 0.0025, 0.0032, 

In [153]:
attn = exclude_cls_sep(attn_per_token)

In [155]:
# zero out the special tokens, [CLS] and [SEP]
# Side note: Multi-sentence input may have [SEP] in the middle of the input. Then the code below should change.

attn_per_token[0] = 0.0 # idx 0 for [CLS]
attn_per_token[torch.nonzero(attn_per_token)[-1]] = 0.0 # last nonzero idx for [SEP]
attn_per_token /= attn_per_token.sum() # normalize over the remanings

In [158]:
attn_per_token

tensor([0.0000, 0.0059, 0.0062, 0.0058, 0.0061, 0.0064, 0.0068, 0.0070, 0.0070,
        0.0065, 0.0065, 0.0062, 0.0033, 0.0041, 0.0031, 0.0038, 0.0041, 0.0035,
        0.0037, 0.0027, 0.0033, 0.0033, 0.0027, 0.0036, 0.0054, 0.0027, 0.0057,
        0.0068, 0.0073, 0.0070, 0.0071, 0.0066, 0.0068, 0.0067, 0.0080, 0.0074,
        0.0070, 0.0070, 0.0072, 0.0062, 0.0071, 0.0070, 0.0071, 0.0071, 0.0066,
        0.0065, 0.0040, 0.0047, 0.0037, 0.0075, 0.0064, 0.0061, 0.0061, 0.0068,
        0.0059, 0.0057, 0.0059, 0.0075, 0.0053, 0.0059, 0.0062, 0.0062, 0.0063,
        0.0060, 0.0060, 0.0060, 0.0049, 0.0061, 0.0066, 0.0044, 0.0042, 0.0064,
        0.0068, 0.0071, 0.0062, 0.0067, 0.0058, 0.0036, 0.0028, 0.0057, 0.0050,
        0.0079, 0.0076, 0.0032, 0.0030, 0.0031, 0.0082, 0.0080, 0.0075, 0.0078,
        0.0074, 0.0072, 0.0066, 0.0049, 0.0057, 0.0062, 0.0069, 0.0046, 0.0081,
        0.0087, 0.0086, 0.0060, 0.0030, 0.0030, 0.0027, 0.0026, 0.0026, 0.0026,
        0.0025, 0.0024, 0.0026, 0.0033, 

In [160]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

In [105]:
# attention score per word, computed by summing scores for tokens in the word
words = kr3[0]['Review'].split(' ')
attn_per_word = torch.zeros(len(words))
tokens_per_word = []

# start slicing index by 1
# why not 0? Because first token is the [CLS]. cf) We don't have [SEP] in the middle of the input, so we do not consider that.
i = 1

for j, word in enumerate(words):
    # number of tokens in the word
    num_tokens = len(tokenizer.tokenize(word))

    # slice the attention by num_tokens
    tokens_per_word.append(attn_per_token[i:i+num_tokens])
    # sum over tokens to obtain attention per word
    sum = tokens_per_word[j].sum()
    attn_per_word[j] = sum
    i += num_tokens

assert attn_per_token[i+1] == 0.0 # attn score after all the indexing should be zero (corresponding to [SEP] after normalization)
    

In [165]:
attn_per_word = get_attn_per_word(attn)

NameError: name 'get_attn_per_word' is not defined

In [106]:
for attn_, word in zip(attn_per_word, words):
    print(attn_, word)

tensor(0.0120) 숙성
tensor(0.0182) 돼지고기
tensor(0.0397) 전문점입니다.
tensor(0.0073) 건물
tensor(0.0069) 모양
tensor(0.0040) 때문에
tensor(0.0072) 매장
tensor(0.0092) 모양도
tensor(0.0026) 좀
tensor(0.0172) 특이하지만
tensor(0.0210) 쾌적한
tensor(0.0136) 편이고
tensor(0.0134) 살짝
tensor(0.0223) 레트로
tensor(0.0203) 감성으로
tensor(0.0210) 분위기
tensor(0.0135) 잡아놨습니다.
tensor(0.0064) 모든
tensor(0.0322) 직원분들께서
tensor(0.0128) 전부
tensor(0.0173) 가능하다고
tensor(0.0127) 멘트
tensor(0.0243) 쳐주시며,
tensor(0.0119) 고기는
tensor(0.0108) 초반
tensor(0.0211) 커팅까지는
tensor(0.0387) 구워주십니다.
tensor(0.0064) 가격
tensor(0.0186) 저렴한
tensor(0.0075) 편
tensor(0.0092) 아니지만
tensor(0.0160) 맛은
tensor(0.0364) 준수합니다.
tensor(0.0281) 등심덧살이
tensor(0.0167) 인상
tensor(0.0145) 깊었는데
tensor(0.0087) 구이로
tensor(0.0078) 별로일
tensor(0.0025) 줄
tensor(0.0082) 알았는데
tensor(0.0133) 육향
tensor(0.0161) 짙고
tensor(0.0110) 얇게
tensor(0.0061) 저며
tensor(0.0032) 뻑뻑하지
tensor(0.0256) 않았습니다.
tensor(0.0390) 하이라이트는
tensor(0.0319) 된장찌개.
tensor(0.0153) 진짜
tensor(0.0311) 굿입니다.
tensor(0.0136) 버터
tensor(0.02

In [108]:
ratio = 0.2
rationale_idxs = attn_per_word.argsort(descending=True)[:int(len(attn_per_word)*ratio)]
rationale_idxs

tensor([60,  2, 46, 26, 32, 18, 47, 49, 33, 51, 45, 22])

In [133]:
rationale_len = len(rationale_idxs)
best_contin_score = 0.0

for i in range(len(attn_per_word) - rationale_len):
    contin_score = attn_per_word[i:i+rationale_len].sum()
    if contin_score > best_contin_score:
        best_contin_score = contin_score
        best_contin_rationale = words[i:i+rationale_len]

In [135]:
best_contin_rationale, best_contin_score

(['않았습니다.',
  '하이라이트는',
  '된장찌개.',
  '진짜',
  '굿입니다.',
  '버터',
  '간장밥,',
  '골뱅이',
  '국수',
  '등',
  '나중에',
  '더'],
 tensor(0.2483))

In [109]:
rationale = [] # words in the rationale, ordered (for sufficiency eval)
unrationale = [] # words not in the rationale, ordered (for completeness evla)

for i, word in enumerate(words):
    if i in rationale_idxs:
        rationale.append(word)
    else:
        unrationale.append(word)

In [110]:
rationale_text = ' '.join(rationale) # words in the list concated to a sentence
unrationale_text = ' '.join(unrationale)

print(rationale_text, '\n')
print(unrationale_text, '\n')
print(kr3[0]['Review'])

전문점입니다. 직원분들께서 쳐주시며, 구워주십니다. 준수합니다. 등심덧살이 않았습니다. 하이라이트는 된장찌개. 굿입니다. 간장밥, 남겨뒀습니다. 

숙성 돼지고기 건물 모양 때문에 매장 모양도 좀 특이하지만 쾌적한 편이고 살짝 레트로 감성으로 분위기 잡아놨습니다. 모든 전부 가능하다고 멘트 고기는 초반 커팅까지는 가격 저렴한 편 아니지만 맛은 인상 깊었는데 구이로 별로일 줄 알았는데 육향 짙고 얇게 저며 뻑뻑하지 진짜 버터 골뱅이 국수 등 나중에 더 맛봐야 할 것들은 

숙성 돼지고기 전문점입니다. 건물 모양 때문에 매장 모양도 좀 특이하지만 쾌적한 편이고 살짝 레트로 감성으로 분위기 잡아놨습니다. 모든 직원분들께서 전부 가능하다고 멘트 쳐주시며, 고기는 초반 커팅까지는 구워주십니다. 가격 저렴한 편 아니지만 맛은 준수합니다. 등심덧살이 인상 깊었는데 구이로 별로일 줄 알았는데 육향 짙고 얇게 저며 뻑뻑하지 않았습니다. 하이라이트는 된장찌개. 진짜 굿입니다. 버터 간장밥, 골뱅이 국수 등 나중에 더 맛봐야 할 것들은 남겨뒀습니다.


In [111]:
from transformers import BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=2)


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [115]:
tokenizer([rationale_text])

{'input_ids': [[101, 9665, 25934, 34907, 58303, 48345, 119, 9707, 14279, 37712, 27023, 118683, 12424, 9755, 87281, 21406, 117, 8908, 69592, 16323, 119085, 48345, 119, 9691, 15891, 33188, 48345, 119, 9121, 71013, 118786, 106249, 10739, 9523, 119118, 119081, 48345, 119, 9952, 10739, 17342, 41620, 11018, 9099, 13890, 119245, 21789, 119, 8915, 58303, 48345, 119, 8845, 13890, 118969, 117, 8987, 89045, 118808, 119081, 48345, 119, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [117]:
input_tensors = {k:torch.tensor(v) for k,v in tokenizer([rationale_text]).items()}
output = model(**input_tensors)

In [124]:
torch.softmax(logit, dim=-1)

tensor([0.0015, 0.9985])

In [127]:
torch.softmax(output.logits, dim=-1)

tensor([[0.4205, 0.5795]], grad_fn=<SoftmaxBackward0>)

## .py

In [164]:
from fresh import *